In [1]:
import pandas as pd

In [29]:
sheets_dict = pd.read_excel('SI2.xlsx', sheet_name=None)


all_sheets = []
df = pd.DataFrame()
for name, sheet in sheets_dict.items():
    sheet = sheet.iloc[3:, :7]
    sheet["component"] = name
    df = pd.concat([df, sheet])
    #print()

In [30]:
df.to_excel("metals_db.xlsx")

In [2]:
from premise.activity_maps import InventorySet, get_mapping
from premise import *
import brightway2 as bw
bw.projects.set_current("new")

In [3]:
ndb = NewDatabase(
        scenarios = [
            {"model":"image", "pathway":"SSP2-Base", "year":2020,},
            #{"model":"image", "pathway":"SSP2-Base", "year":2050,},
        ],        
        source_db="ecoinvent 3.8 cutoff",
        source_version="3.8",
        key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
    )

premise v.(1, 4, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [4]:
inv = InventorySet(ndb.database)

In [5]:
fp = "/Users/romain/GitHub/premise/premise/data/materials_emission_factors/materials_mapping.yml"
mapping = get_mapping(filepath=fp, var="ecoinvent_aliases")
i = inv.generate_sets_from_filters(mapping)

In [8]:
d = {}
for k, v in i.items():
    for _v in v:
        d[_v] = k

In [11]:
for ds in ndb.scenarios[0]["database"]:
    if d.get(ds["name"]):
        ds["has_variable"] = d[ds["name"]]

In [13]:
pd.DataFrame([[ds["name"], ds["reference product"], ds["location"], ds.get("has_variable")] for ds in ndb.scenarios[0]["database"]],
            columns=["name", "ref prod", "location", "variables"]).to_excel("db_coverage.xlsx")